# 📬 Email Automation for Food Insecurity (Jupyter Notebook Version)

This notebook sends personalized emails and checks for 'yes' responses from patients.

- Sends emails using Gmail SMTP
- Checks inbox for replies
- Saves updated response to a new `.xlsx` file
- Supports optional auto-scheduler (runs `check_responses()` every 10 minutes)

In [ ]:
# ✅ Setup - Imports & Config
import pandas as pd
import smtplib
import imaplib
import email
from email.message import EmailMessage
from datetime import datetime
import os
import time
from threading import Thread

# ✉️ Email credentials
EMAIL_ADDRESS = "ug3460707@gmail.com"
EMAIL_PASSWORD = ""
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
IMAP_SERVER = "imap.gmail.com"
EXCEL_FILE = "sample.xlsx"

In [48]:
def load_patients():
    try:
        df = pd.read_excel(EXCEL_FILE)
        print("[INFO] Excel file loaded successfully.")
        return df
    except Exception as e:
        print(f"[ERROR] Failed to load Excel file: {e}")
        return pd.DataFrame()


def save_patients(df, prefix="updated"):
    try:
        base, ext = os.path.splitext(EXCEL_FILE)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        new_file = f"{prefix}_{base}{ext}"
        df.to_excel(new_file, index=False)
        print(f"[INFO] Data saved to {os.path.abspath(new_file)}")
    except Exception as e:
        print(f"[ERROR] Failed to save Excel file: {e}")


In [55]:
def send_emails():
    df = load_patients()
    if df.empty:
        print("[ERROR] No data found in Excel.")
        return

    sent_count = 0
    fail_count = 0

    try:
        print("[INFO] Connecting to SMTP server...")
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT, timeout=10) as smtp:
            smtp.starttls()
            smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            print("[✅] Logged in to SMTP")

            for _, row in df.iterrows():
                name = row.get("Name", "there")
                to_email = str(row.get("Email", "")).strip()

                if not to_email or "@" not in to_email:
                    print(f"[⚠️] Skipping invalid or missing email for {name}: '{to_email}'")
                    fail_count += 1
                    continue

                msg = EmailMessage()
                msg["Subject"] = "Important: Food Support Inquiry"
                msg["From"] = EMAIL_ADDRESS
                msg["To"] = to_email
                msg.set_content(f"""Hi {name},

We're reaching out to support your food needs. Please reply "yes" to this message if any of the following statements apply to you:

• In the last 12 months, have you worried whether your food would run out before you had money to buy more?
• Did the food you bought just not last, and you didn't have money to get more?

Your answers will help us connect you with local food support resources.

Best regards,
Baylor Scott Nutrition Team
""")

                try:
                    smtp.send_message(msg)
                    print(f"[📤] Sent to {name} <{to_email}>")
                    sent_count += 1
                    time.sleep(1)  # prevent spam-like sending
                except Exception as e:
                    print(f"[❌] Failed to send to {to_email}: {e}")
                    fail_count += 1

        print(f"\n[📊] Summary: {sent_count} sent, {fail_count} failed.")
        print("[✅] Email sending completed.")
    except Exception as e:
        print(f"[❌] SMTP connection error: {e}")


In [56]:
send_emails()

[INFO] Excel file loaded successfully.
[INFO] Connecting to SMTP server...
[✅] Logged in to SMTP
[📤] Sent to Test Chan <teilioteaser@gmail.com>

[📊] Summary: 1 sent, 0 failed.
[✅] Email sending completed.


In [59]:
def check_responses():
    df = load_patients()
    if df.empty:
        print("[ERROR] No patient data loaded.")
        return

    df["Food Insecurity"] = df.get("Food Insecurity", "").astype(str)

    try:
        print("[INFO] Connecting to IMAP...")
        mail = imaplib.IMAP4_SSL(IMAP_SERVER)
        mail.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        mail.select("inbox")

        print("[INFO] Searching for unseen replies...")
        status, messages = mail.search(None, '(UNSEEN SUBJECT "Important: Food Support Inquiry")')
        if status != "OK" or not messages[0]:
            print("[INFO] No new replies found.")
            mail.logout()
            return

        print(f"[INFO] Found {len(messages[0].split())} new replies.")
        for num in messages[0].split():
            status, data = mail.fetch(num, "(RFC822)")
            if status != "OK":
                print(f"[ERROR] Failed to fetch message {num}")
                continue

            raw_email = data[0][1]
            msg = email.message_from_bytes(raw_email)
            sender = email.utils.parseaddr(msg["From"])[1]
            body = ""

            if msg.is_multipart():
                for part in msg.walk():
                    if part.get_content_type() == "text/plain" and "attachment" not in str(part.get("Content-Disposition")):
                        body = part.get_payload(decode=True).decode(errors="ignore")
                        break
            else:
                body = msg.get_payload(decode=True).decode(errors="ignore")

            # Extract only the top reply line before any quoted text
            clean_response = body.split("\n")[0].strip().lower()
            print(f"[REPLY] {sender} ➜ \"{clean_response}\"")

            # Determine and log result
            positive_keywords = ["yes", "ok", "yeah"]
            if any(keyword in clean_response for keyword in positive_keywords):
                df.loc[df["Email"].str.lower() == sender.lower(), "Food Insecurity"] = "Yes"
                print(f"[✅] Marked YES for {sender}")
            else:
                df.loc[df["Email"].str.lower() == sender.lower(), "Food Insecurity"] = "No"
                print(f"[INFO] Marked NO for {sender}")


        save_patients(df, prefix="updated")
        mail.logout()
        print("[✅] Response check complete.")
    except Exception as e:
        print(f"[❌ IMAP ERROR] {e}")


In [62]:
check_responses()

[INFO] Excel file loaded successfully.
[INFO] Connecting to IMAP...
[INFO] Searching for unseen replies...
[INFO] Found 1 new replies.
[REPLY] teilioteaser@gmail.com ➜ "ok no problem"
[✅] Marked YES for teilioteaser@gmail.com
[INFO] Data saved to /home/youj/Projects/TaskSoft/twilio_test/with_email/updated_sample.xlsx
[✅] Response check complete.


In [38]:
# 🔁 Optional Scheduler (Auto-check every 10 minutes)
def auto_check_scheduler(interval_minutes=10):
    print(f"[SCHEDULER] Starting auto-check every {interval_minutes} minutes...")
    while True:
        try:
            check_responses()
        except Exception as e:
            print(f"[SCHEDULER ERROR] {e}")
        time.sleep(interval_minutes * 60)